In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split
tqdm.pandas()
train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)
train[:10].question_text
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', 
          "'",  '&', '/', '[', ']', '>', '<', '%', '=', '#', '+', 
          '\\',  '§', '″', '′','¿','═']

def preprocessing(sentence):
    """
    Applies preprocessing such as cleaning to a sentence
    Edited from https://www.kaggle.com/hengzheng/pytorch-starter
    
    Args:
    * sentence: Original sentence to be cleaned
    
    Returns:
    Cleaned sentence
    """
    
    # Convert to lower case
    sentence = sentence.lower()
    
    # Add spaces around special characters
    for punct in puncts:
        sentence = sentence.replace(punct, f' {punct} ')
    
    # Replace numbers by hash characters
    sentence = re.sub('[0-9]{5,}', '#####', sentence)
    sentence = re.sub('[0-9]{4}', '####', sentence)
    sentence = re.sub('[0-9]{3}', '###', sentence)
    sentence = re.sub('[0-9]{2}', '##', sentence)
    
    return sentence
train, val = train_test_split(train, test_size=0.1, random_state=2019)
train
train.question_text = train.question_text.apply(preprocessing)
test.question_text = test.question_text.apply(preprocessing)
val.question_text = val.question_text.apply(preprocessing)
train.question_text[0]
from tensorflow.keras.preprocessing.text import Tokenizer

max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)

# Tokenize the sentences
tokenizer = Tokenizer(num_words = max_features, # how many unique words to use 
                     filters='' # characters that will be filtered from the texts
                     )

tokenizer.fit_on_texts(list(train.question_text) + list(test.question_text)+list(val.question_text))

X2 = tokenizer.texts_to_sequences(val.question_text)
X = tokenizer.texts_to_sequences(train.question_text)
X_test = tokenizer.texts_to_sequences(test.question_text)

# Create vocabulary from all words

In [ ]:
vocabulary = tokenizer.word_index
print(f"Tokenized sample tex: {X[0]}\n")
print({word: vocabulary[word] for word in (train.iloc[0].question_text).split(' ')[:-1]})

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = 100 # 745

X2_embed = pad_sequences(X2, 
                  maxlen=MAX_LENGTH, # Max. number of words in sentence
                  padding='pre' # Where to add padding
                 )

X_embed = pad_sequences(X, 
                  maxlen=MAX_LENGTH, # Max. number of words in sentence
                  padding='pre' # Where to add padding
                 )

X_test_embed = pad_sequences(X_test, 
                       maxlen=MAX_LENGTH, # Max. number of words in sentence
                       padding='pre' # Where to add padding
                      )
X_embed

In [ ]:
train[:10].question_text

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model
from tensorflow.compat.v1.keras.layers import CuDNNGRU 
from keras.layers import Bidirectional, GlobalMaxPool1D
inp = Input(shape=(100,))
x = Embedding(max_features, 300)(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
ty2= val['target'].values
ty= train['target'].values
model.fit(X_embed, ty, batch_size=512, epochs=2,validation_data=(X2_embed, ty2))

In [ ]:
pred_y = model.predict([X_test_embed], batch_size=1024, verbose=1)
pred_y1 = model.predict([X_test_embed], batch_size=1024, verbose=1)
pred_y2 = model.predict([X_test_embed], batch_size=1024, verbose=1)

In [ ]:
pred_test_y = 0.33*pred_y + 0.33*pred_y1 + 0.34*pred_y2
pred_test_y = (pred_test_y>0.35).astype(int)
out_df = pd.DataFrame({"qid":test["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)